### Importing Libraries
Import the required libraries for data analysis.

In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import snntorch as snn
from snntorch import utils

In [2]:
class GraphDataset(Dataset):
    FILE_PATH = ''
    
    def __init__(self, user_id=None, label_list=None, num_samples=None, num_steps=None):
        self.user_id = user_id
        self.label_list = label_list
        self.num_samples = num_samples
        self.num_steps = num_steps
        
        self.data, self.labels = self.load_data(user_id, label_list, num_samples)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.Tensor(self.data[idx]), self.labels[idx]
    
    def load_data(self, user_id, label_list, num_samples):
        data = list()
        labels = list()
        
        for i in range(len(label_list)):
            for j in range(1, num_samples + 1):
                try:
                    adjMatrix = scipy.io.loadmat(
                        self.FILE_PATH + 'tmp_graph_user' + str(user_id) +
                        '_label' + str(label_list[i]) +
                        '_item' + str(j) + '.mat')
                    
                except Exception as e:
                    print(f"Load fail: {e}")
                    continue
                
                tmpAdjMatrix = list()
                for k in range(self.num_steps):
                    tmpAdjMatrix.append(adjMatrix['fullMatrix'][:, :, k])
                                        
                data.append(np.array(tmpAdjMatrix))
                labels.append(label_list[i])
                                
        if not data or not labels:
            raise RuntimeError("No data or labels loaded.")
                    
        data = torch.tensor(np.stack(data), dtype=torch.float32)
                
        label_dict = {label: index for index, label in enumerate(self.label_list)}
        new_label_list = [label_dict[label] for label in labels]
        labels = torch.tensor(new_label_list, dtype=torch.long)
        
        return data, labels

### Loading Data
Load the full dataset.

In [3]:
user_id = 1
label_list = [769, 770]
num_samples = 72

# Temporal Dynamics
num_steps = 3

In [4]:
full_dataset = GraphDataset(user_id=user_id, label_list=label_list, num_samples=num_samples, num_steps=num_steps)

Create the train and test set.

In [5]:
train_size = int(0.75 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [6]:
batch_size = 6

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)